In [2]:
import requests
import json
import sys

sys.path.append("..")

from common.aidevs_authorize import get_task, get_token, send_answer
from common.utils import get_message
from common.openai_requests import (
    send_chat_completion,
    send_transcription,
    make_request,
)

from common._secrets import MEMORIZE_MAKE_WEBOOK, WHISPER_MAKE_WEBHOOK

In [3]:
BASE_MAKE_URL = "https://hook.eu2.make.com"

webhook_url = f"{BASE_MAKE_URL}/{MEMORIZE_MAKE_WEBOOK}"

resp = requests.post(
    webhook_url,
    headers={"content-type": "application/json"},
    data=json.dumps({"type": "memory", "content": "memory note sunday2"}),
)

In [4]:
token = get_token("whisper")
task = get_task(token)
task

{'code': 0,
 'msg': 'please return transcription of this file: https://zadania.aidevs.pl/data/mateusz.mp3',
 'hint': 'use WHISPER model - https://platform.openai.com/docs/guides/speech-to-text'}

In [5]:
import re

pattern = r"(?P<url>https?://[^\s]+)"
urls_found = re.findall(pattern, task["msg"])

In [6]:
task_file = requests.get(urls_found[0])
audio_downloaded = task_file.content

In [68]:
resp = send_transcription(
    urls_found[0], audio_downloaded, language="PL", model="whisper-1"
)
resp

{'text': 'Cześć! Kiedy ostatnio korzystaliście z sztucznej inteligencji, czy zastanawialiście się nad tym, skąd czerpie ona swoją wiedzę? No pewnie, że tak, inaczej nie byłoby was tutaj na szkoleniu. Ale czy przemyśleliście możliwość dostosowania tej wiedzy do waszych własnych, indywidualnych potrzeb?'}

In [71]:
send_answer(token, resp["text"])

'{\n    "code": 0,\n    "msg": "OK",\n    "note": "CORRECT"\n}'

In [7]:
whisper_webhook_url = f"{BASE_MAKE_URL}/{WHISPER_MAKE_WEBHOOK}"

In [13]:
# NIE DZIALA

resp = requests.post(whisper_webhook_url, files={"file": ("test", audio_downloaded)})
resp

<Response [500]>

In [14]:
resp.text

'Scenario failed to complete.'